In [33]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, to_date, year
from pyspark.sql.functions import split, array_distinct, concat_ws

spark = SparkSession.builder \
    .appName("Patent") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0") \
    .getOrCreate()


In [ ]:
spark

In [35]:

df = spark.read.format("com.mongodb.spark.sql.DefaultSource") \
    .option("database", "FPO_Patent")\
    .option("collection", "FPO_DATA")\
    .option("uri", "mongodb://127.0.0.1:27017/FPO_Patent.DATA") \
    .option('inferSchema','true')\
    .load()


In [ ]:

df.columns

In [ ]:
df.printSchema()

In [ ]:

df.show(5)

In [ ]:
df.count()

In [ ]:

df.select(['Title','Country']).show()


# Cleaning


In [ ]:
import pandas as pd

data = pd.read_csv("C:\\Users\\ADMIN\Downloads\\projet\patent_data_FPO (1).csv")
data.head()

In [ ]:
data.isna().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
duplicate_rows_count = df.count() - df.distinct().count() 
print(duplicate_rows_count)


In [ ]:
data=data.to_csv("patent_data_FPO.csv", index=False)

# EDA


In [45]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

df.createOrReplaceTempView("patents")

### 1- Count the number of patents by country:

In [ ]:
patent_counts_by_country = df.groupBy('Country').count()
patent_counts_by_country.show()


In [ ]:
patent_counts_by_country_sql = spark.sql("""
    SELECT Country, COUNT(*) AS Patent_Count
    FROM patents
    GROUP BY Country
    ORDER BY Patent_Count DESC
""")
patent_counts_by_country_sql.show()

### 2- Find the most common inventors:

#### *count the number of patents per inventor, treating each inventor individually.

In [ ]:
from pyspark.sql.functions import regexp_extract, expr
from pyspark.sql.functions import split, explode, count


exploded_inventors = df.withColumn("Inventor", explode(split(df['Inventors'], ', ')))

patents_per_inventor = exploded_inventors.groupBy("Inventor").agg(count("Patent Number").alias("Number of Patents"))
patents_per_inventor.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

patents_per_inventor = spark.sql("""
    SELECT inventor, COUNT(*) AS num_patents
    FROM (
        SELECT explode(split(Inventors, ', ')) AS inventor
        FROM patents
    )
    GROUP BY inventor
    ORDER BY num_patents DESC
""")

patents_per_inventor.show()


#### *count the number of patents per inventor

In [ ]:
patent_counts_by_inventory = df.groupBy('Inventors').count().orderBy('count', ascending=False)
patent_counts_by_inventory.show()


### 3- Get the number of patents published each year:

In [ ]:
from pyspark.sql.functions import year

df_year = df.withColumn('Publication_Year', year(to_date(df['Publication Date'], 'MM/dd/yyyy')))
patents_per_year = df_year.groupBy('Publication_Year').count().orderBy('Publication_Year',ascending=False)
patents_per_year.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


patents_year_df = patents_per_year.toPandas().sort_values(by='Publication_Year', ascending=True)
sns.barplot(x='Publication_Year', y='count', data=patents_year_df)
plt.xticks(rotation=90, fontsize=7)  
plt.show()


### 4- Find the top assignees (companies) with the most patents:


In [ ]:
assignee_counts = df.groupBy('Assignee').count().orderBy('count', ascending=False)
assignee_counts.show()

In [ ]:
patents_assignees_df=assignee_counts.toPandas()
patents_assignees_df = patents_assignees_df.sort_values(by='count', ascending=False)

patents_assignees_top10 = patents_assignees_df.head(10)
sns.barplot(x='Assignee', y='count', data=patents_assignees_top10)
plt.xlabel('Assignee')
plt.ylabel('Count')
plt.xticks(rotation=90)  
plt.show()

### 5- Find patents with specific keywords in the title

In [ ]:
specific_keywords_title = spark.sql("""
    SELECT *
    FROM patents
    WHERE LOWER(Title) LIKE '%electric vehicle battery%'
""")

specific_keywords_title.show()


### 6-visualize the number of patents with "Electric Vehicle Battery" in the title by year

In [ ]:
specific_keywords_title = spark.sql("""
    SELECT YEAR(to_date(`Publication Date`, 'MM/dd/yyyy')) AS Publication_Year, COUNT(*) AS Patent_Count
    FROM patents
    WHERE LOWER(Title) LIKE '%electric vehicle battery%'
    GROUP BY Publication_Year
    ORDER BY Publication_Year
""")

specific_keywords_title_df = specific_keywords_title.toPandas()

plt.figure(figsize=(10, 6))
plt.plot(specific_keywords_title_df['Publication_Year'], specific_keywords_title_df['Patent_Count'], marker='o')
plt.xlabel('Publication Year')
plt.ylabel('Number of Patents')
plt.title('Number of Patents with "Electric Vehicle Battery" in Title by Year')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


### 7-The most common keywords/terms used in the title of patents

In [ ]:
from pyspark.sql.functions import split, explode, lower, regexp_replace, col, collect_list
from pyspark.ml.feature import StopWordsRemover
import warnings
warnings.filterwarnings("ignore")

df_keywords = df.withColumn("Title", lower(col("Title"))) \
                .withColumn("Title", regexp_replace(col("Title"), "[^a-zA-Z\s]", "")) \
                .withColumn("TitleWords", split(col("Title"), " "))
df_keywords = df_keywords.withColumn("Keyword", explode(col("TitleWords")))

df_grouped = df_keywords.groupBy("Patent Number").agg(collect_list("Keyword").alias("Keywords"))

remover = StopWordsRemover(inputCol="Keywords", outputCol="CleanKeywords")
df_cleaned = remover.transform(df_grouped).select("CleanKeywords")
df_cleaned = df_cleaned.withColumn("CleanKeyword", explode(col("CleanKeywords")))
keyword_counts = df_cleaned.groupBy("CleanKeyword").count()

top_keywords = keyword_counts.orderBy(keyword_counts["count"].desc()).limit(20)
top_keywords.show()




In [ ]:
import matplotlib.pyplot as plt

top_keywords_df = top_keywords.toPandas()

plt.figure(figsize=(10, 6))
plt.bar(top_keywords_df['CleanKeyword'], top_keywords_df['count'])
plt.xlabel('Keyword')
plt.ylabel('Frequency')
plt.title('Top Keywords in Patent Titles')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


### 7-The most common keywords/terms used in the summary of patents


In [ ]:
from pyspark.sql.functions import split, explode, lower, regexp_replace, col, collect_list
from pyspark.ml.feature import StopWordsRemover

df_summary_keywords = df.withColumn("Summary", lower(col("Summary"))) \
                        .withColumn("Summary", regexp_replace(col("Summary"), "[^a-zA-Z\s]", "")) \
                        .withColumn("SummaryWords", split(col("Summary"), " "))
df_summary_keywords = df_summary_keywords.withColumn("Keyword", explode(col("SummaryWords")))
df_grouped_summary = df_summary_keywords.groupBy("Patent Number").agg(collect_list("Keyword").alias("SummaryKeywords"))
remover_summary = StopWordsRemover(inputCol="SummaryKeywords", outputCol="CleanSummaryKeywords")
df_cleaned_summary = remover_summary.transform(df_grouped_summary).select("CleanSummaryKeywords")
df_cleaned_summary = df_cleaned_summary.withColumn("CleanSummaryKeyword", explode(col("CleanSummaryKeywords")))
keyword_counts_summary = df_cleaned_summary.groupBy("CleanSummaryKeyword").count()
top_keywords_summary = keyword_counts_summary.orderBy(keyword_counts_summary["count"].desc()).limit(20)
top_keywords_summary.show()


In [ ]:
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
from wordcloud import WordCloud

top_keywords_summary_df = top_keywords_summary.toPandas()

word_freq_dict = dict(zip(top_keywords_summary_df['CleanSummaryKeyword'], top_keywords_summary_df['count']))

wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq_dict)

plt.figure(figsize=(30, 30))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Top Keywords in Summary')
plt.show()
